In [263]:
import numpy as np
import cvxopt
import pandas as pd
import matplotlib as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix

In [264]:
def linear(x, z):
    return np.dot(x, z.T)


def polynomial(x, z, p=5):
    return (1 + np.dot(x, z.T)) ** p


def gaussian(x, z, sigma=0.1):
    return np.exp(-np.linalg.norm(x - z, axis=1) ** 2 / (2 * (sigma ** 2)))

In [265]:
'''class SVM:
    def __init__(self, kernel=gaussian, C=1):
        self.kernel = kernel
        self.C = C

    def fit(self, X, y):
        self.y = y
        self.X = X
        m, n = X.shape
        #print("x=",self.X,sep='\n')
        #print("y=",self.y,sep='\n')
        # Calculate Kernel
        self.K = np.zeros((m, m))
        for i in range(m):
            self.K[i] = self.kernel(X[i,np.newaxis], self.X)
        #print('k=',self.K,sep='\n')
        # Solve with cvxopt final QP needs to be reformulated
        # to match the input form for cvxopt.solvers.qp
        P = cvxopt.matrix(np.outer(y, y) * self.K)
        #print("P=",P,sep='\n')
        q = cvxopt.matrix(-np.ones((m, 1)))
        #print("Q=",q,sep='\n')
        G = cvxopt.matrix(np.vstack((np.eye  (m) * -1, np.eye(m))))
        #print('g=',G,sep='\n')
        h = cvxopt.matrix(np.hstack((np.zeros(m), np.ones(m) * self.C)))
        #print('h=',h,sep= '\n')
        A = cvxopt.matrix(y, (1,m),"d")
        #print('A=',A)
        #for i in A:
        #    print(i)
        b = cvxopt.matrix(np.zeros(1))
        #print('b=',b)
        cvxopt.solvers.options["show_progress"] = False
        sol = cvxopt.solvers.qp(P, q, G, h, A, b)
        #print('sol=',sol,sep='\n')
        self.alphas = np.array(sol["x"])
        #print('alphas=',self.alphas,sep='\n')
        a=self.alphas
        sv = a > 1e-5
        ind1 = np.arange(len(a))
        ind=list()
        for i in ind1:
            if(i in sv):
                ind.append(i)
        ind=np.array(ind)
        self.a = a[sv]
        self.sv = self.X[sv]
        self.sv_y = self.y[sv]
        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * K[ind[n], sv])
        self.b /= len(self.a)
        # Weights
        self.w = np.zeros(n_features)
        for n in range(len(self.a)):
            self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        
    def project(self, X):
        return np.dot(X, self.w) + self.b
    
    
    def predict(self, X):
        return np.sign(self.project(X))'''

'class SVM:\n    def __init__(self, kernel=gaussian, C=1):\n        self.kernel = kernel\n        self.C = C\n\n    def fit(self, X, y):\n        self.y = y\n        self.X = X\n        m, n = X.shape\n        #print("x=",self.X,sep=\'\n\')\n        #print("y=",self.y,sep=\'\n\')\n        # Calculate Kernel\n        self.K = np.zeros((m, m))\n        for i in range(m):\n            self.K[i] = self.kernel(X[i,np.newaxis], self.X)\n        #print(\'k=\',self.K,sep=\'\n\')\n        # Solve with cvxopt final QP needs to be reformulated\n        # to match the input form for cvxopt.solvers.qp\n        P = cvxopt.matrix(np.outer(y, y) * self.K)\n        #print("P=",P,sep=\'\n\')\n        q = cvxopt.matrix(-np.ones((m, 1)))\n        #print("Q=",q,sep=\'\n\')\n        G = cvxopt.matrix(np.vstack((np.eye  (m) * -1, np.eye(m))))\n        #print(\'g=\',G,sep=\'\n\')\n        h = cvxopt.matrix(np.hstack((np.zeros(m), np.ones(m) * self.C)))\n        #print(\'h=\',h,sep= \'\n\')\n        A = cvxopt

In [266]:
'''class SVM:
    def fit(self, X, y):
        n_samples, n_features = X.shape
        # P = X^T X
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = np.dot(X[i], X[j])
        P = cvxopt.matrix(np.outer(y, y) * K)
        # q = -1 (1xN)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        # A = y^T 
        A = cvxopt.matrix(y, (1, n_samples),"d")
        # b = 0 
        b = cvxopt.matrix(0.0)
        # -1 (NxN)
        G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
        # 0 (1xN)
        h = cvxopt.matrix(np.zeros(n_samples))
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        # Lagrange have non zero lagrange multipliers
        sv = a > 1e-5
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]
        # Intercept
        self.b = 0.0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * K[ind[n], sv])
        self.b /= len(self.a)
        # Weights
        self.w = np.zeros(n_features)
        for n in range(len(self.a)):
            self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        
    def project(self, X):
        return np.dot(X, self.w) + self.b
    
    
    def predict(self, X):
        return np.sign(self.project(X))'''

'class SVM:\n    def fit(self, X, y):\n        n_samples, n_features = X.shape\n        # P = X^T X\n        K = np.zeros((n_samples, n_samples))\n        for i in range(n_samples):\n            for j in range(n_samples):\n                K[i,j] = np.dot(X[i], X[j])\n        P = cvxopt.matrix(np.outer(y, y) * K)\n        # q = -1 (1xN)\n        q = cvxopt.matrix(np.ones(n_samples) * -1)\n        # A = y^T \n        A = cvxopt.matrix(y, (1, n_samples),"d")\n        # b = 0 \n        b = cvxopt.matrix(0.0)\n        # -1 (NxN)\n        G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))\n        # 0 (1xN)\n        h = cvxopt.matrix(np.zeros(n_samples))\n        solution = cvxopt.solvers.qp(P, q, G, h, A, b)\n        # Lagrange multipliers\n        a = np.ravel(solution[\'x\'])\n        # Lagrange have non zero lagrange multipliers\n        sv = a > 1e-5\n        ind = np.arange(len(a))[sv]\n        self.a = a[sv]\n        self.sv = X[sv]\n        self.sv_y = y[sv]\n        # Intercept\n  

In [304]:
class SVM:
    def __init__(self, kernel=gaussian, C=0.001):
        self.kernel = kernel
        self.C = C

    def fit(self, X, y):
        self.y = y
        self.X = X
        m, n = X.shape
        #print("x=",self.X,sep='\n')
        #print("y=",self.y,sep='\n')
        # Calculate Kernel
        self.K = np.zeros((m, m))
        for i in range(m):
            self.K[i] = self.kernel(X[i,np.newaxis], self.X)
        #print('k=',self.K,sep='\n')
        # Solve with cvxopt final QP needs to be reformulated
        # to match the input form for cvxopt.solvers.qp
        P = cvxopt.matrix(np.outer(y, y) * self.K)
        #print("P=",P,sep='\n')
        q = cvxopt.matrix(-np.ones((m, 1)))
        #print("Q=",q,sep='\n')
        G = cvxopt.matrix(np.vstack((np.eye  (m) * -1, np.eye(m))))
        #print('g=',G,sep='\n')
        h = cvxopt.matrix(np.hstack((np.zeros(m), np.ones(m) * self.C)))
        #print('h=',h,sep= '\n')
        A = cvxopt.matrix(y, (1,m),"d")
        #print('A=',A)
        #for i in A:
        #    print(i)
        b = cvxopt.matrix(np.zeros(1))
        #print('b=',b)
        cvxopt.solvers.options["show_progress"] = False
        sol = cvxopt.solvers.qp(P, q, G, h, A, b)
        #print('sol=',sol,sep='\n')
        self.alphas = np.array(sol["x"])
        #print('alphas=',self.alphas,sep='\n')

    def predict(self, X):
        y_predict = np.zeros((X.shape[0]))
        sv = self.get_parameters(self.alphas)
        for i in range(X.shape[0]):
            y_predict[i] = np.sum(
                self.alphas[sv]
                * self.y[sv, np.newaxis]
                * self.kernel(X[i], self.X[sv])[:, np.newaxis]
            )
        #print(y_predict)
        #print(self.b)
        return np.sign(y_predict + self.b)

    def get_parameters(self, alphas):
        threshold = 1e-5

        sv = ((alphas > threshold) * (alphas < self.C)).flatten()
        self.w = np.dot(self.X[sv].T, alphas[sv] * self.y[sv, np.newaxis])
        self.b = np.mean(
            self.y[sv, np.newaxis]
            - self.alphas[sv] * self.y[sv, np.newaxis] * self.K[sv, sv][:, np.newaxis]
        )
        #print(sv)
        return sv


In [317]:
from sklearn.datasets import load_breast_cancer
data=load_breast_cancer()
features=pd.DataFrame(data.data,columns=data.feature_names)
target=pd.DataFrame(data.target,columns=['target'])
features=features.drop(['mean compactness','mean concave points','worst fractal dimension','area error','compactness error','concave points error','concavity error','fractal dimension error','mean concavity','mean symmetry','perimeter error','radius error','symmetry error','texture error','worst concavity','worst smoothness','worst symmetry'],axis=1)
target=np.where(target<=0,-1,1)
xtrain,xtest,ytrain,ytest=train_test_split(features,target,test_size=0.2)

svm = SVM()
svm.fit(np.array(xtrain),np.array(ytrain) )
ypred = svm.predict(np.array(xtest))
#print(y_pred)
#print(ytest)
#plot_contour(np.array(features), np.array(target), svm)

#print(f"Accuracy: {sum(ytest==y_pred)/ytest.shape[0]}")

In [318]:
mean_squared_error(ypred,ytest)

1.6140350877192982

In [319]:
r2_score(ypred,ytest)

0.0

In [308]:
confusion_matrix(ypred,ytest)

array([[ 0,  0],
       [40, 74]], dtype=int64)

# sklearn

In [309]:
from sklearn.svm import SVC

In [310]:
model=SVC()
model.fit(xtrain,ytrain)
ypreds=model.predict(xtest)

C:\Users\Bhagwan\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [311]:
mean_squared_error(ypreds,ytest)

0.24561403508771928

In [312]:
r2_score(ypreds,ytest)

0.7199017199017199

In [313]:
confusion_matrix(ypreds,ytest)

array([[35,  2],
       [ 5, 72]], dtype=int64)

In [277]:
data=pd.concat([pd.DataFrame(ypred,columns=["my"]),pd.DataFrame(ypreds,columns=["sk"]),pd.DataFrame(ytest,columns=["test"])],axis=1)
data[:50]

,my,sk,test
0,1.0,-1,-1
1,1.0,1,1
2,1.0,1,-1
3,1.0,1,1
4,1.0,-1,-1
5,1.0,1,1
6,1.0,1,1
7,1.0,1,1
8,1.0,1,-1
9,1.0,-1,-1


In [278]:
mean_squared_error(ypreds,ypred)

1.087719298245614